### A2.5.1. MLIR Dialects

> *A dialect in MLIR is a logical grouping of operations, types, and attributes under a namespace, representing a specific level of abstraction or domain.*

**Explanation:**

**MLIR** (Multi-Level Intermediate Representation) extends the LLVM philosophy by allowing multiple IRs (dialects) to coexist in one framework, rather than forcing everything into a single flat IR.

A **dialect** defines:
- **Operations** — the instructions of that abstraction level (e.g., `arith.addi`, `linalg.matmul`).
- **Types** — data representations (e.g., `memref<4x4xf32>`, `tensor<8xf64>`).
- **Attributes** — compile-time metadata attached to operations.

**Key Built-in Dialects:**

| Dialect | Purpose | Example Op |
|---------|---------|------------|
| `arith` | Integer and float arithmetic | `arith.addi`, `arith.mulf` |
| `func` | Function definitions and calls | `func.func`, `func.call` |
| `memref` | Memory-backed multi-dimensional buffers | `memref.alloc`, `memref.load` |
| `tensor` | Value-semantic multi-dimensional data | `tensor.extract`, `tensor.cast` |
| `scf` | Structured control flow (for, if, while) | `scf.for`, `scf.if` |
| `linalg` | Linear algebra operations on tensors/buffers | `linalg.matmul`, `linalg.generic` |
| `affine` | Polyhedral loop/memory analysis | `affine.for`, `affine.load` |
| `llvm` | One-to-one mapping to LLVM IR | `llvm.add`, `llvm.call` |

**Compilation proceeds by progressively lowering** from high-level dialects (linalg, scf) through mid-level dialects (memref, arith) down to the `llvm` dialect, which maps directly to LLVM IR.

**Example:**

```mlir
func.func @add(%a: f32, %b: f32) -> f32 {
  %result = arith.addf %a, %b : f32
  return %result : f32
}
```

This function uses two dialects: `func` (for the function structure) and `arith` (for the addition).

In [ ]:
from dataclasses import dataclass, field


@dataclass
class MLIRType:
    name: str


@dataclass
class MLIROperation:
    dialect: str
    name: str
    operand_types: list[MLIRType]
    result_type: MLIRType

    @property
    def qualified_name(self):
        return f"{self.dialect}.{self.name}"


@dataclass
class Dialect:
    name: str
    operations: list[MLIROperation] = field(default_factory=list)
    types: list[MLIRType] = field(default_factory=list)

    def register_operation(self, op_name, operand_types, result_type):
        operation = MLIROperation(self.name, op_name, operand_types, result_type)
        self.operations.append(operation)
        return operation


f32 = MLIRType("f32")
i32 = MLIRType("i32")
memref_type = MLIRType("memref<4x4xf32>")

arith_dialect = Dialect("arith", types=[f32, i32])
addf = arith_dialect.register_operation("addf", [f32, f32], f32)
muli = arith_dialect.register_operation("muli", [i32, i32], i32)

memref_dialect = Dialect("memref", types=[memref_type])
alloc = memref_dialect.register_operation("alloc", [], memref_type)
load = memref_dialect.register_operation("load", [memref_type, i32], f32)

all_dialects = [arith_dialect, memref_dialect]

for dialect in all_dialects:
    print(f"Dialect: {dialect.name}")
    print(f"  Types: {[t.name for t in dialect.types]}")
    for operation in dialect.operations:
        operand_names = [t.name for t in operation.operand_types]
        print(f"  Op: {operation.qualified_name}({', '.join(operand_names)}) -> {operation.result_type.name}")
    print()

**References:**

[📘 MLIR Project. *MLIR Language Reference.*](https://mlir.llvm.org/docs/LangRef/)

[📘 Lattner, C. et al. (2021). *MLIR: Scaling Compiler Infrastructure for Domain Specific Computation.* IEEE CGO.](https://ieeexplore.ieee.org/document/9370308)

---

[⬅️ Previous: Loop Vectorization](../04_Optimization_Topics/03_loop_vectorization.ipynb) | [Next: Pattern Rewriting ➡️](./02_pattern_rewriting.ipynb)